In [1]:
excluded_session = 3

In [2]:
%env WANDB_PROJECT = final_model_

env: WANDB_PROJECT=final_model_


In [3]:
model_ckp = "bert-base-uncased"
batch_size = 32

In [4]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.3 MB/s eta 0:00:00


In [5]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [6]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=62fa60c22fd774ae51fe4763732123fc909b25e27edd353072a116bf57675a80
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [7]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text_dataset")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating Session1 split:   0%|          | 0/1085 [00:00<?, ? examples/s]

Generating Session2 split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Generating Session3 split:   0%|          | 0/1151 [00:00<?, ? examples/s]

Generating Session4 split:   0%|          | 0/1031 [00:00<?, ? examples/s]

Generating Session5 split:   0%|          | 0/1241 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/minoosh___parquet/minoosh--IEMOCAP_Text_dataset-87e9e0bcfcf0a760/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckp)

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["transcription"], truncation=True)

In [12]:
dataset=dataset.remove_columns(['TURN_NAME'])

In [13]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
labels = dataset["Session1"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["1"]

'hap'

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckp, num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [19]:
tokenized_dataset = tokenized_dataset.rename_column("emotion", "label")

In [20]:
ds1 = tokenized_dataset['Session1'].train_test_split(test_size=0.2)
ds2 = tokenized_dataset['Session2'].train_test_split(test_size=0.2)
ds3 = tokenized_dataset['Session3'].train_test_split(test_size=0.2)
ds4 = tokenized_dataset['Session4'].train_test_split(test_size=0.2)
ds5 = tokenized_dataset['Session5'].train_test_split(test_size=0.2)

In [21]:
import torch
train_ds = torch.utils.data.ConcatDataset([ds1['train'], 
                                           ds2['train'],
                                           #ds3['train'],
                                           ds4['train'],
                                           ds5['train']])

In [22]:
val_ds = torch.utils.data.ConcatDataset([ds1['test'],
                                         ds2['test'],
                                         #ds3['test'],
                                         ds4['test'],
                                         ds5['test']])

In [23]:
training_args = TrainingArguments(
    output_dir="finetuned_bert-base-on-IEMOCAP_3",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=10,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_bert_base_on_IEMOCAP_3'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_3 into local empty directory.


In [24]:
import torch
torch.cuda.empty_cache()

In [25]:
results = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.255300,1.225602,0.463554
2,0.893500,0.921065,0.635535
3,0.718900,0.776068,0.710706
4,0.567600,0.803140,0.720957
5,0.359800,0.845352,0.730068
6,0.277400,0.887119,0.715262
7,0.269900,0.937218,0.717540
8,0.196300,0.968577,0.724374
9,0.256500,1.009867,0.701595
10,0.184900,1.042518,0.717540


In [26]:
trainer.evaluate(tokenized_dataset['Session3'])

{'eval_loss': 1.0341538190841675,
 'eval_accuracy': 0.6490008688097306,
 'eval_runtime': 4.0122,
 'eval_samples_per_second': 286.878,
 'eval_steps_per_second': 8.973,
 'epoch': 20.0}

In [27]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_3
   dcf08eb..1069937  main -> main

   dcf08eb..1069937  main -> main

To https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_3
   1069937..5281d72  main -> main

   1069937..5281d72  main -> main



'https://huggingface.co/minoosh/finetuned_bert-base-on-IEMOCAP_3/commit/10699378a1a87039a9fdd44fce99669850126241'